In [2]:
import os
import copy
import time
import random
from tqdm import tqdm

import PIL
from PIL import Image

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold

import torch
import torchvision
from torchvision import datasets, transforms, models

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score
# fix seeds
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 2019
seed_everything(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.enabled = False
print(device)

cuda


### KFOLD
- 5 folds for ensemble model

In [3]:
df_train = pd.read_csv('../data/train.csv')
df_train.replace(196, 0, inplace=True)
df_train['fold'] = 999

skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=False)
for index, (_, val_index) in enumerate(skf.split(df_train['img_file'], df_train['class'])):
    df_train['fold'].iloc[val_index] = index

/home/preser9191/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
def prepare_fold_dataset(df_train, fold):
    X_train = df_train[df_train['fold']!=fold]['img_file']
    X_val = df_train[df_train['fold']==fold]['img_file']
    y_train = df_train[df_train['fold']!=fold]['class']
    y_val = df_train[df_train['fold']==fold]['class']
    
    return X_train.values, X_val.values, y_train.values, y_val.values

In [5]:
for i in range(5):
    X_train, X_val, y_train, y_val = prepare_fold_dataset(df_train, 0)
    print(X_train.shape)
    print(y_train.shape)
    print(X_val.shape)
    print(y_val.shape)
    print("-"*8)
    

(7914,)
(7914,)
(2076,)
(2076,)
--------
(7914,)
(7914,)
(2076,)
(2076,)
--------
(7914,)
(7914,)
(2076,)
(2076,)
--------
(7914,)
(7914,)
(2076,)
(2076,)
--------
(7914,)
(7914,)
(2076,)
(2076,)
--------


### Prepare Dataset

In [6]:
# Batch size
batch_size = 64

# Image resize
image_size = (224, 224)

# Datapath
TRAIN_DATA_PATH = '../data/train_crop/'
TEST_DATA_PATH = '../data/test_crop/'

# torch dataset
class TrainImages(Dataset):
    def __init__(self, images, labels, mode=None, transforms=None):
        self.images = images
        self.labels = labels
        self.mode = mode
        self.transforms = transforms[self.mode]
        
    def __len__(self):
        return self.images.shape[0]
        
    def __getitem__(self, idx):
        image = Image.open(TRAIN_DATA_PATH + self.images[idx]).convert("RGB")
        image = self.transforms(image)
        label = self.labels[idx]
        
        return image, label
    
    
class TestImages(Dataset):
    def __init__(self, images, mode=None, transforms=None):
        self.images = images
        self.mode = mode
        self.transforms = transforms[self.mode]
        
    def __len__(self):
        return self.images.shape[0]
        
    def __getitem__(self, idx):
        image = Image.open(TEST_DATA_PATH + self.images[idx]).convert("RGB")
        image = self.transforms(image)
        
        return image
    
    
transform = {
    'train': transforms.Compose([
        transforms.Resize(image_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225])
    ])
}

In [7]:
def prepare_dataloader(X_train, X_val, y_train, y_val):
    train_dataset = TrainImages(images=X_train, labels=y_train, mode='train', transforms=transform)
    val_dataset = TrainImages(images=X_val, labels=y_val, mode='val', transforms=transform)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    dataloaders = {
        'train': train_dataloader,
        'val': val_dataloader
    }

    dataset_sizes = {
        'train': len(train_dataset),
        'val': len(val_dataset)
    }
    return dataloaders, dataset_sizes

### Train

In [8]:
def train(model, dataloader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    
    for batch_index, (batch_image, batch_target) in enumerate(dataloader):
        batch_image, batch_target = batch_image.to(device), batch_target.to(device)
        
        # forward pass
        optimizer.zero_grad()
        batch_output = model(batch_image)
        loss = criterion(batch_output, batch_target)
        
        # backward
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() / len(dataloader)
        if batch_index % (len(dataloader)//3) == 0:
            print(". . . batch: {}, batch_loss: {}".format(batch_index, loss.item()))
    return train_loss

def validate(model, dataloader, criterion, dataset_sizes):
    model.eval()
    
    val_preds = np.zeros((dataset_sizes['val'], 1))
    val_loss = 0.0

    with torch.no_grad():
        for batch_index, (batch_image, batch_target) in enumerate(dataloader):
            batch_image, batch_target = batch_image.to(device), batch_target.to(device)
            
            batch_output = model(batch_image).detach()
            _, batch_preds = torch.max(batch_output, 1)
            
            loss = criterion(batch_output, batch_target)
            val_loss += loss.item() / len(dataloader)
            
            val_preds[batch_index * batch_size: (batch_index+1) * batch_size] = batch_preds.cpu().view(-1, 1).numpy()

        val_score = f1_score(y_val, val_preds, average='micro')
        
    return val_loss, val_score

def train_model(model, dataloaders, lr, dataset_sizes, num_epochs, PATH):
    
    best_score = 0.0
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    results = {
        'best_epoch': 0,
        'best_score': 0,
    }
    
    for epoch in range(num_epochs):
        print("{} / {}".format(epoch+1, num_epochs))
        train_loss = train(model, dataloaders['train'], criterion, optimizer)
        val_loss, val_score = validate(model, dataloaders['val'], criterion, dataset_sizes)
        
        print("EPOCH: {}, train_loss: {}, val_loss: {}, val_score: {}".format(epoch+1, train_loss, val_loss, val_score))
        if val_score > best_score:
            best_score = val_score
            torch.save(model.state_dict(), './{}/best_model_epoch_{}_score_{}.pt'.format(PATH, epoch+1, val_score))
            results['best_epoch'] = epoch+1
            results['best_score'] = best_score
            print(">>>>>>  EPOCH {}: validation score {}".format(epoch+1, val_score))
    
    return results

In [12]:
model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)

In [13]:
# 0번째 fold
fold = 0

X_train, X_val, y_train, y_val = prepare_fold_dataset(df_train, fold)
dataloaders, dataset_sizes = prepare_dataloader(X_train, X_val, y_train, y_val)

model_res.to(device)

PATH = 'fold{}/'.format(fold) + 'rough'
results = train_model(model_res, dataloaders, 0.001, dataset_sizes, 5, PATH)
print(results)

1 / 5
. . . batch: 0, batch_loss: 5.383825302124023
. . . batch: 41, batch_loss: 5.299849033355713
. . . batch: 82, batch_loss: 5.233278751373291
. . . batch: 123, batch_loss: 5.312289714813232
EPOCH: 1, train_loss: 5.291198522813858, val_loss: 5.228442495519465, val_score: 0.01878612716763006
>>>>>>  EPOCH 1: validation score 0.01878612716763006
2 / 5
. . . batch: 0, batch_loss: 5.282308101654053
. . . batch: 41, batch_loss: 5.1560540199279785
. . . batch: 82, batch_loss: 5.07704496383667
. . . batch: 123, batch_loss: 5.177755355834961
EPOCH: 2, train_loss: 5.137968890128598, val_loss: 5.100189252333208, val_score: 0.0197495183044316
>>>>>>  EPOCH 2: validation score 0.0197495183044316
3 / 5
. . . batch: 0, batch_loss: 5.020870685577393
. . . batch: 41, batch_loss: 4.926645755767822
. . . batch: 82, batch_loss: 5.113419532775879
. . . batch: 123, batch_loss: 4.968969345092773
EPOCH: 3, train_loss: 5.035809605352338, val_loss: 5.145202521121862, val_score: 0.016377649325626204
4 / 5
. 

In [16]:
fold = 0

model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('./fold0/rough/best_model_epoch_5_score_0.03901734104046243.pt'))

model_res.to(device)

PATH = 'fold{}/'.format(fold) + 'rough'
results = train_model(model_res, dataloaders, 0.001, dataset_sizes, 10, PATH)
print(results)

1 / 10
. . . batch: 0, batch_loss: 4.246837615966797
. . . batch: 41, batch_loss: 4.343574523925781
. . . batch: 82, batch_loss: 4.252584934234619
. . . batch: 123, batch_loss: 4.106612205505371
EPOCH: 1, train_loss: 4.250045176475279, val_loss: 4.246904828331687, val_score: 0.07996146435452794
>>>>>>  EPOCH 1: validation score 0.07996146435452794
2 / 10
. . . batch: 0, batch_loss: 4.149881839752197
. . . batch: 41, batch_loss: 3.9386115074157715
. . . batch: 82, batch_loss: 3.77221417427063
. . . batch: 123, batch_loss: 3.844247341156006
EPOCH: 2, train_loss: 3.908779055841508, val_loss: 3.8655671784372045, val_score: 0.10886319845857419
>>>>>>  EPOCH 2: validation score 0.10886319845857419
3 / 10
. . . batch: 0, batch_loss: 3.5502612590789795
. . . batch: 41, batch_loss: 3.8402698040008545
. . . batch: 82, batch_loss: 3.587817668914795
. . . batch: 123, batch_loss: 3.235092878341675
EPOCH: 3, train_loss: 3.611608159157541, val_loss: 3.884415417006521, val_score: 0.12379576107899808
>

In [17]:
fold = 0

model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('./fold0/rough/best_model_epoch_9_score_0.47832369942196534.pt'))

model_res.to(device)

PATH = 'fold{}/'.format(fold) + 'rough'
results = train_model(model_res, dataloaders, 0.001, dataset_sizes, 10, PATH)
print(results)

1 / 10
. . . batch: 0, batch_loss: 1.6552002429962158
. . . batch: 41, batch_loss: 1.444204330444336
. . . batch: 82, batch_loss: 0.9600077867507935
. . . batch: 123, batch_loss: 1.1740903854370117
EPOCH: 1, train_loss: 1.3083350216188747, val_loss: 1.9234032558672352, val_score: 0.5048169556840078
>>>>>>  EPOCH 1: validation score 0.5048169556840078
2 / 10
. . . batch: 0, batch_loss: 1.0093985795974731
. . . batch: 41, batch_loss: 0.9463107585906982
. . . batch: 82, batch_loss: 1.211683988571167
. . . batch: 123, batch_loss: 1.2292311191558838
EPOCH: 2, train_loss: 1.0798399501269869, val_loss: 1.9095017332019228, val_score: 0.49614643545279385
3 / 10
. . . batch: 0, batch_loss: 1.0029034614562988
. . . batch: 41, batch_loss: 1.1067020893096924
. . . batch: 82, batch_loss: 0.8739738464355469
. . . batch: 123, batch_loss: 1.1196417808532715
EPOCH: 3, train_loss: 0.9829437910549103, val_loss: 1.5341745994307776, val_score: 0.5814065510597303
>>>>>>  EPOCH 3: validation score 0.581406551

In [18]:
fold = 0

model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('./fold0/rough/best_model_epoch_9_score_0.6271676300578035.pt'))

model_res.to(device)

PATH = 'fold{}/'.format(fold) + 'rough'
results = train_model(model_res, dataloaders, 0.001, dataset_sizes, 5, PATH)
print(results)

1 / 5
. . . batch: 0, batch_loss: 0.37338078022003174
. . . batch: 41, batch_loss: 0.32476985454559326
. . . batch: 82, batch_loss: 0.3499756455421448
. . . batch: 123, batch_loss: 0.41411399841308594
EPOCH: 1, train_loss: 0.5039797786983754, val_loss: 1.8053203134825735, val_score: 0.5948940269749519
>>>>>>  EPOCH 1: validation score 0.5948940269749519
2 / 5
. . . batch: 0, batch_loss: 0.37293627858161926
. . . batch: 41, batch_loss: 0.32039737701416016
. . . batch: 82, batch_loss: 0.37813687324523926
. . . batch: 123, batch_loss: 0.3415590524673462
EPOCH: 2, train_loss: 0.4132883915257068, val_loss: 1.3744796911875405, val_score: 0.6551059730250481
>>>>>>  EPOCH 2: validation score 0.6551059730250481
3 / 5
. . . batch: 0, batch_loss: 0.37042543292045593
. . . batch: 41, batch_loss: 0.30435675382614136
. . . batch: 82, batch_loss: 0.445711225271225
. . . batch: 123, batch_loss: 0.5060069561004639
EPOCH: 3, train_loss: 0.37092277083185426, val_loss: 1.4581100145975752, val_score: 0.648

#### lr changed to 0.0001

In [19]:
fold = 0

model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('./fold0/rough/best_model_epoch_5_score_0.6685934489402697.pt'))

model_res.to(device)

PATH = 'fold{}/'.format(fold) + 'rough'
results = train_model(model_res, dataloaders, 0.0001, dataset_sizes, 5, PATH)
print(results)

1 / 5
. . . batch: 0, batch_loss: 0.26603302359580994
. . . batch: 41, batch_loss: 0.11748883128166199
. . . batch: 82, batch_loss: 0.10489501804113388
. . . batch: 123, batch_loss: 0.20275545120239258
EPOCH: 1, train_loss: 0.14286818845017302, val_loss: 0.8847493773156948, val_score: 0.779383429672447
>>>>>>  EPOCH 1: validation score 0.779383429672447
2 / 5
. . . batch: 0, batch_loss: 0.13722552359104156
. . . batch: 41, batch_loss: 0.07206806540489197
. . . batch: 82, batch_loss: 0.15307839214801788
. . . batch: 123, batch_loss: 0.09004052728414536
EPOCH: 2, train_loss: 0.08164021179019924, val_loss: 0.8492354479703037, val_score: 0.785645472061657
>>>>>>  EPOCH 2: validation score 0.785645472061657
3 / 5
. . . batch: 0, batch_loss: 0.038217902183532715
. . . batch: 41, batch_loss: 0.09447608143091202
. . . batch: 82, batch_loss: 0.08864694088697433
. . . batch: 123, batch_loss: 0.1042739674448967
EPOCH: 3, train_loss: 0.06729429539653563, val_loss: 0.927479602170713, val_score: 0.7

In [20]:
fold = 0

model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('./fold0/rough/best_model_epoch_2_score_0.785645472061657.pt'))

model_res.to(device)

PATH = 'fold{}/'.format(fold) + 'rough'
results = train_model(model_res, dataloaders, 0.0001, dataset_sizes, 3, PATH)
print(results)

1 / 3
. . . batch: 0, batch_loss: 0.10267733782529831
. . . batch: 41, batch_loss: 0.051810793578624725
. . . batch: 82, batch_loss: 0.06250932812690735
. . . batch: 123, batch_loss: 0.11724892258644104
EPOCH: 1, train_loss: 0.07666434596983658, val_loss: 0.8477138425364639, val_score: 0.7870905587668593
>>>>>>  EPOCH 1: validation score 0.7870905587668593
2 / 3
. . . batch: 0, batch_loss: 0.10274454951286316
. . . batch: 41, batch_loss: 0.05277039855718613
. . . batch: 82, batch_loss: 0.06517122685909271
. . . batch: 123, batch_loss: 0.12235032021999359
EPOCH: 2, train_loss: 0.06382877607980082, val_loss: 0.9512522459933253, val_score: 0.7764932562620422
3 / 3
. . . batch: 0, batch_loss: 0.08058720827102661
. . . batch: 41, batch_loss: 0.06236293166875839
. . . batch: 82, batch_loss: 0.05246765911579132
. . . batch: 123, batch_loss: 0.04324960708618164
EPOCH: 3, train_loss: 0.05411495618341911, val_loss: 0.8722764127182238, val_score: 0.785645472061657
{'best_epoch': 1, 'best_score': 

#### lr changed to 0.00001

In [21]:
fold = 0

model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('./fold0/rough/best_model_epoch_1_score_0.7870905587668593.pt'))

model_res.to(device)

PATH = 'fold{}/'.format(fold) + 'rough'
results = train_model(model_res, dataloaders, 0.00001, dataset_sizes, 10, PATH)
print(results)

1 / 10
. . . batch: 0, batch_loss: 0.04467710852622986
. . . batch: 41, batch_loss: 0.027035189792513847
. . . batch: 82, batch_loss: 0.11692338436841965
. . . batch: 123, batch_loss: 0.028341684490442276
EPOCH: 1, train_loss: 0.05913375088224002, val_loss: 0.8829545604460166, val_score: 0.7914258188824663
>>>>>>  EPOCH 1: validation score 0.7914258188824663
2 / 10
. . . batch: 0, batch_loss: 0.05197174474596977
. . . batch: 41, batch_loss: 0.06104442477226257
. . . batch: 82, batch_loss: 0.03625480830669403
. . . batch: 123, batch_loss: 0.0462426133453846
EPOCH: 2, train_loss: 0.05352216397201823, val_loss: 0.8743639619964542, val_score: 0.785645472061657
3 / 10
. . . batch: 0, batch_loss: 0.037354402244091034
. . . batch: 41, batch_loss: 0.051488082855939865
. . . batch: 82, batch_loss: 0.04191826656460762
. . . batch: 123, batch_loss: 0.01837806962430477
EPOCH: 3, train_loss: 0.053001852105221445, val_loss: 0.87860474848386, val_score: 0.791907514450867
>>>>>>  EPOCH 3: validation s

In [22]:
fold = 0

model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('./fold0/rough/best_model_epoch_3_score_0.791907514450867.pt'))

model_res.to(device)

PATH = 'fold{}/'.format(fold) + 'rough'
results = train_model(model_res, dataloaders, 0.000001, dataset_sizes, 10, PATH)
print(results)

1 / 10
. . . batch: 0, batch_loss: 0.11710284650325775
. . . batch: 41, batch_loss: 0.03234390914440155
. . . batch: 82, batch_loss: 0.08893438428640366
. . . batch: 123, batch_loss: 0.04258214682340622
EPOCH: 1, train_loss: 0.050638629769485806, val_loss: 0.9253194088285621, val_score: 0.7870905587668593
>>>>>>  EPOCH 1: validation score 0.7870905587668593
2 / 10
. . . batch: 0, batch_loss: 0.06959838420152664
. . . batch: 41, batch_loss: 0.0741816982626915
. . . batch: 82, batch_loss: 0.0527956485748291
. . . batch: 123, batch_loss: 0.08831766992807388
EPOCH: 2, train_loss: 0.051807586749595014, val_loss: 0.9722666853305066, val_score: 0.7842003853564548
3 / 10
. . . batch: 0, batch_loss: 0.04893384873867035
. . . batch: 41, batch_loss: 0.05226784199476242
. . . batch: 82, batch_loss: 0.04128000885248184
. . . batch: 123, batch_loss: 0.0834939256310463
EPOCH: 3, train_loss: 0.05059158180149332, val_loss: 0.9425019495414965, val_score: 0.789980732177264
>>>>>>  EPOCH 3: validation sco

#### lr change to 0.0000001
- with EPOCH: 3, train_loss: 0.053001852105221445, val_loss: 0.87860474848386, val_score: 0.791907514450867

In [23]:
fold = 0

model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('./fold0/rough/best_model_epoch_3_score_0.791907514450867.pt'))

model_res.to(device)

PATH = 'fold{}/'.format(fold) + 'rough'
results = train_model(model_res, dataloaders, 0.0000001, dataset_sizes, 5, PATH)
print(results)

1 / 5
. . . batch: 0, batch_loss: 0.05719216167926788
. . . batch: 41, batch_loss: 0.04256844520568848
. . . batch: 82, batch_loss: 0.05564809590578079
. . . batch: 123, batch_loss: 0.03641868755221367
EPOCH: 1, train_loss: 0.05317618748954228, val_loss: 0.8727182687231991, val_score: 0.7943159922928708
>>>>>>  EPOCH 1: validation score 0.7943159922928708
2 / 5
. . . batch: 0, batch_loss: 0.058001965284347534
. . . batch: 41, batch_loss: 0.0617021769285202
. . . batch: 82, batch_loss: 0.02316904254257679
. . . batch: 123, batch_loss: 0.06976498663425446
EPOCH: 2, train_loss: 0.049729162211259496, val_loss: 0.8796036704020066, val_score: 0.7890173410404624
3 / 5
. . . batch: 0, batch_loss: 0.04677455499768257
. . . batch: 41, batch_loss: 0.060270022600889206
. . . batch: 82, batch_loss: 0.01879439502954483
. . . batch: 123, batch_loss: 0.027426855638623238
EPOCH: 3, train_loss: 0.05180054671701884, val_loss: 0.8766368586908686, val_score: 0.7904624277456648
4 / 5
. . . batch: 0, batch_l

### FOLD 1

In [ ]:
model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('./fold1/rough/best_model_epoch_9_score_0.2787046123650638.pt'))

# 0번째 fold
fold = 1

X_train, X_val, y_train, y_val = prepare_fold_dataset(df_train, fold)
dataloaders, dataset_sizes = prepare_dataloader(X_train, X_val, y_train, y_val)

model_res.to(device)

PATH = 'fold{}/'.format(fold) + 'rough'
results = train_model(model_res, dataloaders, 0.001, dataset_sizes, 17, PATH)
print(results)

1 / 17
. . . batch: 0, batch_loss: 2.213510751724243
. . . batch: 41, batch_loss: 2.493255853652954
. . . batch: 82, batch_loss: 2.096853017807007


### FOLD2

In [ ]:
model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)

# 0번째 fold
fold = 2

X_train, X_val, y_train, y_val = prepare_fold_dataset(df_train, fold)
dataloaders, dataset_sizes = prepare_dataloader(X_train, X_val, y_train, y_val)

model_res.to(device)

PATH = 'fold{}/'.format(fold) + 'rough'
results = train_model(model_res, dataloaders, 0.001, dataset_sizes, 30, PATH)
print(results)

### FOLD3


In [ ]:
model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)

# 0번째 fold
fold = 3

X_train, X_val, y_train, y_val = prepare_fold_dataset(df_train, fold)
dataloaders, dataset_sizes = prepare_dataloader(X_train, X_val, y_train, y_val)

model_res.to(device)

PATH = 'fold{}/'.format(fold) + 'rough'
results = train_model(model_res, dataloaders, 0.001, dataset_sizes, 30, PATH)
print(results)

### FOLD4

In [ ]:
model_res = models.resnet101(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)

# 0번째 fold
fold = 4

X_train, X_val, y_train, y_val = prepare_fold_dataset(df_train, fold)
dataloaders, dataset_sizes = prepare_dataloader(X_train, X_val, y_train, y_val)

model_res.to(device)

PATH = 'fold{}/'.format(fold) + 'rough'
results = train_model(model_res, dataloaders, 0.001, dataset_sizes, 30, PATH)
print(results)